# Pre-Trained tokenizer 사용하기   
BPE Tokenizer는 구글의 Sentencepiece를 많이 사용한다(Sentencepiece 사용법 : https://github.com/google/sentencepiece)

In [6]:
# 이미 학습된 Tokenizer와 vocabulary 가져오기
from transformers import BertTokenizer
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')  # bert-base-uncased라는 모델을 사용하기 위해 Tokenizer을 일치시키기 위해 미리 학습된 Tokenizer가져옴

print(len(tokenizer.vocab))

30522


In [11]:
# 토크니저가 문장을 Tokenization하는 방법
tokenizer = BertTokenizer.from_pretrained('bert-base-uncased')
sentence = 'My dog is cute. He likes playing'
print(tokenizer.tokenize(sentence))                                       # tokenization 결과 보기

tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-uncased')
print(len(tokenizer.vocab))
print(tokenizer.tokenize(sentence))                                       # 

sentence = '나는 책상 위에 사과를 먹었다. 알고 보니 그 사과는 Jason 것이었다. 그래서 Jason에게 사과를 했다'
print(tokenizer.tokenize(sentence))

['my', 'dog', 'is', 'cute', '.', 'he', 'likes', 'playing']
105879
['my', 'dog', 'is', 'cut', '##e', '.', 'he', 'likes', 'playing']
['나는', 'ᄎ', '##ᅢᆨ', '##상', '위에', 'ᄉ', '##ᅡ', '##과', '##를', 'ᄆ', '##ᅥ', '##ᆨ', '##었다', '.', '알', '##고', 'ᄇ', '##ᅩ', '##니', '그', 'ᄉ', '##ᅡ', '##과', '##는', 'jason', '것이', '##었다', '.', '그', '##래', '##서', 'jason', '##에게', 'ᄉ', '##ᅡ', '##과', '##를', '했다']
